In [3]:
import pandas as pd
import numpy as np
import copy

In [45]:
data = pd.read_csv("store_data.csv",header=None)

numCustomers = data.shape[0]
ALL_ITEMS = pd.unique(data[[i for i in range(20)]].values.ravel('K'))
#print(numCustomers)
#print(data.shape)
print(data[0][0])

# WARNING: DATA BASED CODE below to remove nan
ALL_ITEMS = np.delete(ALL_ITEMS,-6)   # removing nan
#ALL_ITEMS.shape
#ALL_ITEMS

shrimp


In [28]:
# FIXING parameters
minSupport = 0.05
minConfidence = 0.2

In [6]:
def support(items,data,customers=numCustomers):
    i = 1
    count = 0
    # can also use set()
    for index, prods in data.iterrows():
        i = 1
        for item in items:
            if item not in prods.as_matrix():
                i = 0
                break
        if i:
            count += 1
    return count/customers
        

In [7]:
def confidence(antecedent,consequent,customers=numCustomers):
    numerator = 0
    denominator = 0
    is_antecedent = 1
    is_concequent = 1
    for index, prods in data.iterrows():
        is_antecedent = 1
        is_concequent = 1
        for item in antecedent:
            if item not in prods.as_matrix():
                is_antecedent = is_concequent = 0
                break
        for item in consequent:
            if item not in prods.as_matrix():
                is_concequent = 0
                break
        if is_antecedent:
            denominator += 1
            if is_concequent:
                numerator += 1
    return numerator/denominator

In [8]:
def findFrequentItemsets(ALL_ITEMS,data):
    frequent = []
    lst = []
    #with tqdm(total=len(ALL_ITEMS)) as pbar:
    for item in ALL_ITEMS:
        lst = [item]
        if support([item],data) >= minSupport:
            frequent.append([item])
    return frequent

In [9]:
def generateCandidates(items):
    candidates = []
    for i in range(len(items)):
        for j in range(i,len(items)):
            if all(items[i]) not in items[j]:
                candidates.append(items[i] + list(set(items[j]) - set(items[i])))
    return candidates

In [36]:
freq_items1 = findFrequentItemsets(ALL_ITEMS,data)

counter = 1
i = 1
l = 0
# finding frequent itemsets
while counter == 1 and i <= 25:
    i += 1
    cand = generateCandidates(freq_items1)
    k = 0
    l = 0
    tmp = []
    counter = 0
    for j in cand:
        if support(j,data) >= minSupport:
            tmp.append(j)
            l += 1
            # if found new frequent itemset of length i
            if len(j) == i:
                counter = 1
        k += 1
    tmp = sorted(tmp)
    freq_items1 = [tmp[i] for i in range(len(tmp)) if i == 0 or tmp[i] != tmp[i-1]]

In [37]:
len(freq_items1)
freq_items1 = sorted(freq_items1)
freq_items1 = [freq_items1[i] for i in range(len(freq_items1)) if i == 0 or freq_items1[i] != freq_items1[i-1]]
for i in range(len(freq_items1)):
    print(freq_items1[i],end='\t')   
    print(support(freq_items1[i],data))

['burgers']	0.0871883748833489
['cake']	0.08105585921877083
['chicken']	0.05999200106652446
['chocolate']	0.1638448206905746
['chocolate', 'mineral water']	0.05265964538061592
['cookies']	0.08038928142914278
['cooking oil']	0.0510598586855086
['eggs']	0.17970937208372217
['eggs', 'mineral water']	0.05092654312758299
['escalope']	0.0793227569657379
['french fries']	0.1709105452606319
['frozen smoothie']	0.06332489001466471
['frozen vegetables']	0.09532062391681109
['grated cheese']	0.0523930142647647
['green tea']	0.13211571790427942
['ground beef']	0.09825356619117451
['low fat yogurt']	0.07652313024930009
['milk']	0.12958272230369283
['mineral water']	0.23836821757099053
['mineral water', 'chocolate']	0.05265964538061592
['mineral water', 'eggs']	0.05092654312758299
['mineral water', 'spaghetti']	0.05972536995067324
['olive oil']	0.0658578856152513
['pancakes']	0.09505399280095987
['shrimp']	0.07145713904812692
['soup']	0.05052659645380616
['spaghetti']	0.17411011865084655
['tomatoes'

In [21]:
anti = []
conse = []
def createRules(items):
    res = []
    for i in items[0]:
        if i not in items[1]:
            conse = [j for j in items[1]]
            temp = []
            elem = []
            elem.append(i)
            temp.append(list(set(items[0])-set(elem)))
            temp.append(conse)
            temp[1].append(i)
            res.append(temp)
    #print(res)
    return res

In [22]:
generateCandidates([['shrimp'],['burgers'],['turkey']])

[['shrimp'],
 ['shrimp', 'burgers'],
 ['shrimp', 'turkey'],
 ['burgers'],
 ['burgers', 'turkey'],
 ['turkey']]

In [38]:
freq_1 = freq_items1
#print(freq_1)
test1 = [ i for i in freq_1 if len(i) > 1]
test = [ [i,[]] for i in test1]
#for i in test:
#    print(i)
rules = []
final_rules = []
for i in test:
    rules.append(createRules(i))
#or i in range(len(rules)):
#   print(rules[i])
result_anticedent = []
result_consequent = []
for i in range(len(rules)):
    if confidence(rules[i][0],rules[i][1]) >= minConfidence:
        result_anticedent.append(rules[i][0])
        result_consequent.append(rules[i][1])

found = 1
result_anticedent = []
result_consequent = []
temp = []
while found:
    found = 0
    rules = []
    temp = []
    for i in range(len(test)):
        rules = createRules(test[i])
        for j in range(len(rules)):
            if confidence(rules[j][0],rules[j][1]) >= minConfidence:
                temp.append(rules[j])
                result_anticedent.append(rules[j][0])
                result_consequent.append(rules[j][1])
                found = 1
    test = temp

#len(rules)
#print(result_anticedent)
#print(result_consequent)
confidence(['mineral water'],['chocolate'])
confidence(['chocolate'],['mineral water'])
for i in range(len(result_anticedent)):
    print(str(result_anticedent[i]) + " --> " + str(result_consequent[i]))

['mineral water'] --> ['chocolate']
['chocolate'] --> ['mineral water']
['mineral water'] --> ['eggs']
['eggs'] --> ['mineral water']
['chocolate'] --> ['mineral water']
['mineral water'] --> ['chocolate']
['eggs'] --> ['mineral water']
['mineral water'] --> ['eggs']
['spaghetti'] --> ['mineral water']
['mineral water'] --> ['spaghetti']


In [39]:
print(confidence(['mineral water'],['chocolate']))
print(confidence(['chocolate'],['mineral water']))
print(confidence(['mineral water'],['eggs']))
print(confidence(['eggs'],['mineral water']))
print(confidence(['chocolate'],['mineral water']))
print(confidence(['mineral water'],['chocolate']))
print(confidence(['eggs'],['mineral water']))
print(confidence(['mineral water'],['eggs']))
print(confidence(['spaghetti'],['mineral water']))
print(confidence(['mineral water'],['spaghetti']))

0.220917225950783
0.3213995117982099
0.21364653243847875
0.28338278931750743
0.3213995117982099
0.220917225950783
0.28338278931750743
0.21364653243847875
0.3430321592649311
0.2505592841163311
